## [seurat-scanpy相互转化](https://www.jianshu.com/p/f1d5268bc96e)
R进行SCT后保存
```R
library(SeuratDisk)
library(Seurat)
source("../scProcess_SCT.R")
options(Seurat.object.assay.version = "v3")            ## 默认v5, SaveH5Seurat会出错

counts <- Read10X(data.dir = "pbmc3k/",gene.column = 2)   ## 1:ENSG00xxx   2:geneName
pbmc3k <- CreateSeuratObject(counts,  min.cells=3, min.features = 200)
pbmc3k <- scProcess_SCT(pbmc3k)

SaveH5Seurat(pbmc3k, filename = 'pbmc3k.h5Seurat')
Convert('pbmc3k.h5Seurat', dest = "h5ad")              ## pbmc3k.h5ad


save_loom <- SeuratDisk::as.loom(x = pbmc3k, filename = 'pbmc3k.loom', verbose = FALSE)
save_loom$close_all()
```
Python进行读取，[Anndata操作](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html)

```py
adata = sc.read_loom('tmp/pbmc3k.loom', sparse=True, cleanup=False)
adata = sc.read_h5ad("tmp/pbmc3k.h5ad")    ## FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances']
```

In [1]:
import scanpy as sc
import loompy
import numpy as np
adata = sc.read_loom('tmp/pbmc3k.loom', sparse=True, cleanup=False)
adata

AnnData object with n_obs × n_vars = 2697 × 12571
    obs: 'CC.Difference', 'G2M.Score', 'Phase', 'S.Score', 'SCT_snn_res.0.1', 'nCount_RNA', 'nCount_SCT', 'nFeature_RNA', 'nFeature_SCT', 'old.ident', 'orig.ident', 'pct.hb', 'pct.mt', 'pct.rp', 'seurat_clusters'
    layers: 'counts'

In [2]:
adata.X   ## print(adata.X)      ## 等同于 counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx').T.tocsc()

<2697x12571 sparse matrix of type '<class 'numpy.float32'>'
	with 2188133 stored elements in Compressed Sparse Row format>

In [3]:
adata.to_df(layer="counts")

Gene,AL627309.1,RP11-206L10.2,LINC00115,NOC2L,KLHL17,PLEKHN1,HES4,ISG15,AGRN,C1orf159,...,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC145212.1,AL592183.1,AL354822.1,PNRC2.1,SRSF10.1
CellID,,,,,,,,,,,,,,,,,,,,,
AAACATACAACCAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
AAACATTGAGCTAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,16.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0
AAACATTGATCAGC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,2.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
AAACCGTGCTTCCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,1.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
AAACCGTGTATGCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCGAACTCTCAT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTCTACTGAGGCA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
TTTCTACTTCCTCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


In [4]:
adata.obs   ## adata.obs["CC.Difference"]

,CC.Difference,G2M.Score,Phase,S.Score,SCT_snn_res.0.1,nCount_RNA,nCount_SCT,nFeature_RNA,nFeature_SCT,old.ident,orig.ident,pct.hb,pct.mt,pct.rp,seurat_clusters
CellID,,,,,,,,,,,,,,,
AAACATACAACCAC-1,0.061106,-0.020841,S,0.040265,1,2419.0,2275.0,779,768,1,1,0.00000,3.017776,43.695742,1
AAACATTGAGCTAC-1,0.021337,-0.029858,G1,-0.008521,3,4903.0,2582.0,1352,1115,1,1,0.00000,3.793596,42.402611,3
AAACATTGATCAGC-1,-0.049305,0.042402,G2M,-0.006903,1,3147.0,2466.0,1129,1110,1,1,0.00000,0.889736,31.680966,1
AAACCGTGCTTCCG-1,0.024803,-0.011546,S,0.013257,2,2639.0,2341.0,960,940,1,1,0.00000,1.743085,24.251610,2
AAACCGTGTATGCG-1,-0.008071,-0.002323,G1,-0.010394,4,980.0,1902.0,521,550,1,1,0.00000,1.224490,14.897959,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCGAACTCTCAT-1,0.032213,-0.043418,G1,-0.011205,2,3459.0,2476.0,1153,1125,1,1,0.02891,2.110437,27.146574,2
TTTCTACTGAGGCA-1,0.052035,-0.041360,S,0.010676,3,3443.0,2479.0,1224,1182,1,1,0.00000,0.929422,33.168748,3
TTTCTACTTCCTCG-1,0.021649,-0.014599,S,0.007050,3,1684.0,1986.0,622,605,1,1,0.00000,2.197150,39.786223,3


In [5]:
adata.var

""
Gene
AL627309.1
RP11-206L10.2
LINC00115
NOC2L
KLHL17
...
AC145212.1
AL592183.1
AL354822.1
